In [1]:
import pandas as pd
import os 
import sys
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
for file in os.listdir():
    print(file)

.git
damir_data.ipynb
Test Survey_Juan.xlsx
test.csv


### Read all sheets in excel file

In [3]:
xl = pd.ExcelFile('Test Survey_Juan.xlsx')

### Loop through sheets

In [4]:
sheet_dict = {}

for sheet in xl.sheet_names:
    df = xl.parse(sheet)
    sheet_dict[sheet] = df

### Define function

In [5]:
def drilling_function(df_inital_data,bin,metadata):
    df_inital_data['INC Rad'] = df_inital_data['Incl'].apply(lambda x: math.radians(x))
    df_inital_data['AZI Rad'] = df_inital_data['Azm (°)'].apply(lambda x: math.radians(x))
    df_inital_data['DLS Rad'] = df_inital_data['DLS'].apply(lambda x: math.radians(x))
    df_inital_data
    
    filter_df = df_inital_data.loc[df_inital_data['MD (ft)']<bin]
    
    filter_df.loc[:,'Weight of Pipe Above']= ((filter_df['MD (ft)'] - filter_df['MD (ft)'].loc[0])*(metadata.get('Pipe(lb/ft)'))+metadata.get('ESP Weight(lbs)'))
    
    filter_df.loc[:,'K'] = (filter_df['Weight of Pipe Above']/(metadata.get('E(psi)')*metadata.get('MOI (in^4)')))**(1/2)
    
    filter_df.loc[:,'tanh eq'] = np.tanh(filter_df['K']*metadata.get('Half-Length(in)'))/(filter_df['K']*metadata.get('Half-Length(in)'))
    
    filter_df.loc[:,'DLS Side Force (psi)'] = (filter_df['DLS']/(filter_df['tanh eq']))*(np.pi/432000)*metadata.get('E(psi)')*metadata.get('OD(in)')
    filter_df.loc[:,'DLS Side Force (lbf)'] = filter_df.loc[:,'DLS Side Force (psi)']*metadata.get('Area(in^2)')
    
    filter_df['Fnormal (total)'] = (filter_df['MD (ft)'])*metadata.get('Pipe(lb/ft)')*np.sin(filter_df['INC Rad'])
        
    filter_df['Fnormal (lbf/joint)'] = filter_df['Fnormal (total)']/(filter_df['MD (ft)']/30)
    filter_df['DLS Side Force (lbf/joint)'] = filter_df['DLS Side Force (lbf)']/(filter_df['MD (ft)'].max()/30)

    filter_df['Total Side Load (lbf/joint)'] = np.abs(filter_df['Fnormal (total)'])
    
    filter_df['Side Load + Inc'] = filter_df['Fnormal (lbf/joint)']+filter_df['DLS Side Force (lbf/joint)']
    
    return filter_df

In [6]:
metadata = {'MOI (in^4)':2.302849714,
'Pipe(lb/ft)' :6.5,
'Area(in^2)':2.857925687,
'OD(in)':	2.875,
'ID(in)': 2.151,
'Half-Length(in)':	180,
'E(psi)':	30000000,
'ESP Weight(lbs)':	5000}

In [7]:
kop = 8100

In [8]:
df_inital_data = list(sheet_dict.values())[1]
df_inital_data

,MD (ft),Incl,Azm (°),TVD (ft),NS (ft),EW (ft),DLS,PKR,KOP
0,25,0.00,0.00,25.00,0.00,0.00,0.00,NaN,NaN
1,154,0.60,14.00,154.00,0.66,0.16,0.47,NaN,NaN
2,241,0.50,22.10,240.99,1.45,0.42,0.15,NaN,NaN
3,329,0.40,135.70,328.99,1.58,0.78,0.86,NaN,NaN
4,420,1.30,168.20,419.98,0.35,1.21,1.08,NaN,NaN
...,...,...,...,...,...,...,...,...,...
210,18870,90.35,0.72,8447.89,9299.15,-265.39,0.68,NaN,NaN
211,18960,90.71,0.55,8447.06,9389.14,-264.40,0.44,NaN,NaN
212,19050,91.05,0.65,8445.67,9479.12,-263.45,0.39,NaN,NaN
213,19088,91.78,0.80,8444.73,9517.11,-262.97,1.96,NaN,NaN


### Define bin

In [9]:
bin = 550

In [10]:
dataframe_dict = {}

for value in np.arange(bin,kop+bin,bin):
    
    df_test = drilling_function(df_inital_data,value,metadata)
    dataframe_dict[value] = df_test[['MD (ft)','Side Load + Inc']]
    

C:\Users\jfhur\AppData\Local\Temp\ipykernel_29072\3684070273.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df.loc[:,'Weight of Pipe Above']= ((filter_df['MD (ft)'] - filter_df['MD (ft)'].loc[0])*(metadata.get('Pipe(lb/ft)'))+metadata.get('ESP Weight(lbs)'))
C:\Users\jfhur\AppData\Local\Temp\ipykernel_29072\3684070273.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df.loc[:,'K'] = (filter_df['Weight of Pipe Above']/(metadata.get('E(psi)')*metadata.get('MOI (in^4)')))**(1/2)
C:\Users\

In [11]:
different_depths_calc_df = pd.concat(dataframe_dict,axis=1)
different_depths_calc_df

550                     1100                    1650                  \
   MD (ft) Side Load + Inc MD (ft) Side Load + Inc MD (ft) Side Load + Inc   
0     25.0        0.000000    25.0        0.000000    25.0        0.000000   
1    154.0       89.582707   154.0       43.782188   154.0       30.359131   
2    241.0       30.653372   241.0       15.506102   241.0       11.066792   
3    329.0      173.089422   329.0       83.242822   329.0       56.910893   
4    420.0      227.362096   420.0      110.722885   420.0       76.538672   
..     ...             ...     ...             ...     ...             ...   
88     NaN             NaN     NaN             NaN     NaN             NaN   
89     NaN             NaN     NaN             NaN     NaN             NaN   
90     NaN             NaN     NaN             NaN     NaN             NaN   
91     NaN             NaN     NaN             NaN     NaN             NaN   
92     NaN             NaN     NaN             NaN     NaN             NaN   

      2200                    2750                  ...    6050  \
   MD (ft) Side Load + Inc MD (ft) Side Load + Inc  ... MD (ft)   
0     25.0        0.000000    25.0        0.000000  ...    25.0   
1    154.0       22.980074   154.0       18.769107  ...   154.0   
2    241.0        8.626371   241.0        7.233709  ...   241.0   
3    329.0       42.435441   329.0       34.174815  ...   329.0   
4    420.0       57.746583   420.0       47.022605  ...   420.0   
..     ...             ...     ...             ...  ...     ...   
88     NaN             NaN     NaN             NaN  ...     NaN   
89     NaN             NaN     NaN             NaN  ...     NaN   
90     NaN             NaN     NaN             NaN  ...     NaN   
91     NaN             NaN     NaN             NaN  ...     NaN   
92     NaN             NaN     NaN             NaN  ...     NaN   

                      6600                    7150                    7700  \
   Side Load + Inc MD (ft) Side Load + Inc MD (ft) Side Load + Inc MD (ft)   
0         0.000000    25.0        0.000000    25.0        0.000000    25.0   
1         9.484261   154.0        8.873017   154.0        8.350133   154.0   
2         4.163000   241.0        3.960847   241.0        3.787918   241.0   
3        15.960789   329.0       14.761715   329.0       13.735975   329.0   
4        23.377084   420.0       21.820442   420.0       20.488823   420.0   
..             ...     ...             ...     ...             ...     ...   
88             NaN     NaN             NaN     NaN             NaN     NaN   
89             NaN     NaN             NaN     NaN             NaN     NaN   
90             NaN     NaN             NaN     NaN             NaN     NaN   
91             NaN     NaN             NaN     NaN             NaN     NaN   
92             NaN     NaN             NaN     NaN             NaN     NaN   

                      8250                  
   Side Load + Inc MD (ft) Side Load + Inc  
0         0.000000      25        0.000000  
1         7.903132     154        7.530644  
2         3.640084     241        3.516894  
3        12.859096     329       12.128390  
4        19.350455     420       18.401850  
..             ...     ...             ...  
88             NaN    8043        8.354554  
89             NaN    8107       75.882413  
90             NaN    8127      431.821489  
91             NaN    8153      349.555935  
92             NaN    8198      447.709440  

[93 rows x 30 columns]

In [12]:
final_value_list = []
index_list = []
for value in np.arange(bin,kop+bin,bin):
    final_value_list.append(different_depths_calc_df[(value,'Side Load + Inc')])
    if value > kop:
       index_list.append(different_depths_calc_df[(value,'MD (ft)')].values)

In [13]:
final_df = pd.concat(final_value_list,axis=1)
final_df.index = index_list[0]

In [14]:
final_dict = {}

for idx, row in final_df.iterrows():
    final_dict[idx] = [np.max(row)]

In [15]:
final_plot_df = pd.DataFrame.from_dict(final_dict).T
final_plot_df.columns = ['Max value']
final_plot_df

,Max value
25,0.000000
154,89.582707
241,30.653372
329,173.089422
420,227.362096
...,...
8043,8.354554
8107,75.882413
8127,431.821489
8153,349.555935


### CSV creation

In [16]:
final_dict = {}

for idx, row in final_df.iterrows():
    final_dict[idx] = [np.max(row)]
    
final_plot_df = pd.DataFrame.from_dict(final_dict).T


In [17]:
final_df

,550,1100,1650,2200,2750,3300,3850,4400,4950,5500,6050,6600,7150,7700,8250
,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc,Side Load + Inc
25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
154,89.582707,43.782188,30.359131,22.980074,18.769107,15.959698,13.951983,12.455311,11.294707,10.224563,9.484261,8.873017,8.350133,7.903132,7.530644
241,30.653372,15.506102,11.066792,8.626371,7.233709,6.304573,5.640577,5.145593,4.761755,4.407834,4.163000,3.960847,3.787918,3.640084,3.516894
329,173.089422,83.242822,56.910893,42.435441,34.174815,28.663613,24.725091,21.789078,19.512329,17.413034,15.960789,14.761715,13.735975,12.859096,12.128390
420,227.362096,110.722885,76.538672,57.746583,47.022605,39.867942,34.754937,30.943393,27.987708,25.262394,23.377084,21.820442,20.488823,19.350455,18.401850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.354554
8107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.882413
8127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431.821489


In [18]:
final_final_plot_df = pd.merge(final_df,final_plot_df, left_index=True,right_index=True)
final_final_plot_df

C:\Users\jfhur\AppData\Local\Temp\ipykernel_29072\243180797.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  final_final_plot_df = pd.merge(final_df,final_plot_df, left_index=True,right_index=True)


,"(550, Side Load + Inc)","(1100, Side Load + Inc)","(1650, Side Load + Inc)","(2200, Side Load + Inc)","(2750, Side Load + Inc)","(3300, Side Load + Inc)","(3850, Side Load + Inc)","(4400, Side Load + Inc)","(4950, Side Load + Inc)","(5500, Side Load + Inc)","(6050, Side Load + Inc)","(6600, Side Load + Inc)","(7150, Side Load + Inc)","(7700, Side Load + Inc)","(8250, Side Load + Inc)",0
25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
154,89.582707,43.782188,30.359131,22.980074,18.769107,15.959698,13.951983,12.455311,11.294707,10.224563,9.484261,8.873017,8.350133,7.903132,7.530644,89.582707
241,30.653372,15.506102,11.066792,8.626371,7.233709,6.304573,5.640577,5.145593,4.761755,4.407834,4.163000,3.960847,3.787918,3.640084,3.516894,30.653372
329,173.089422,83.242822,56.910893,42.435441,34.174815,28.663613,24.725091,21.789078,19.512329,17.413034,15.960789,14.761715,13.735975,12.859096,12.128390,173.089422
420,227.362096,110.722885,76.538672,57.746583,47.022605,39.867942,34.754937,30.943393,27.987708,25.262394,23.377084,21.820442,20.488823,19.350455,18.401850,227.362096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.354554,8.354554
8107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.882413,75.882413
8127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431.821489,431.821489
8153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,349.555935,349.555935


In [19]:
final_final_plot_df.to_csv('test.csv')